## Import Python Libraries

In [1]:
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori, association_rules

## Collecting Data

Dataset berasal dari Rekap Penjualan PT Arma Inti Raya (Banjarmasin) tahun 2017

In [2]:
data = pd.read_csv("Penjualan2017.csv")
print("Data shape", data.shape)

Data shape (1240, 9)


## Explore Dataset

In [3]:
df = data.copy()
df.drop(['Invoice'], axis=1).head()

,No,Tanggal,NamaPerusahaan,NamaBarang,Qty,Sat,HargaSatuan,JumlahHarga
0,1,1/3/2017,PT. BERSAMA SEJAHTERA SAKTI (GAF),OIL SEAL 120MM X 150MM X 14MM,10,PCS,46150,461500
1,2,1/3/2017,PT. BERSAMA SEJAHTERA SAKTI (GAF),OIL SEAL 130MM X 160MM X 13MM,10,PCS,43500,435000
2,3,1/3/2017,PT. BERSAMA SEJAHTERA SAKTI (GAF),OIL SEAL 150MM X 180MM X 14MM,10,PCS,152400,1524000
3,4,1/3/2017,PT. SWADAYA ANDIKA (SLF),CAT HAMPLE TAHAN PANAS 600˚C,10,KG,250000,2500000
4,5,1/3/2017,PT. BERSAMA SEJAHTERA SAKTI (GAF),ATAP TRANSPARAN 0.8MM X 840MM X 6000MM,70,LBR,895000,62650000


In [4]:
# Eksplor kolom data
df.columns

Index(['No', 'Tanggal', 'Invoice', 'NamaPerusahaan', 'NamaBarang', 'Qty',
       'Sat', 'HargaSatuan', 'JumlahHarga'],
      dtype='object')

In [5]:
# Informasi data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   No              1240 non-null   int64  
 1   Tanggal         1240 non-null   object 
 2   Invoice         1231 non-null   float64
 3   NamaPerusahaan  1240 non-null   object 
 4   NamaBarang      1240 non-null   object 
 5   Qty             1240 non-null   int64  
 6   Sat             1240 non-null   object 
 7   HargaSatuan     1240 non-null   int64  
 8   JumlahHarga     1240 non-null   int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 87.3+ KB


In [6]:
# Cek baris data yang tidak memiliki nomor nota
df.isnull().any()

No                False
Tanggal           False
Invoice            True
NamaPerusahaan    False
NamaBarang        False
Qty               False
Sat               False
HargaSatuan       False
JumlahHarga       False
dtype: bool

## Preprocessing Data

In [7]:
# Membersihkan extra white space
df['NamaPerusahaan'] = df['NamaPerusahaan'].str.strip()
df['NamaBarang'] = df['NamaBarang'].str.strip()

# Menghapus baris tanpa nomor Invoice/Nota
df.dropna(axis=0, subset=['Invoice'], inplace = True)
df['Invoice'] = df['Invoice'].astype('str').str.split('.').str[0]

# Memilih kolom yang akan digunakan
df = df[['Invoice', 'NamaPerusahaan', 'NamaBarang', 'Qty']]
df.drop(['Invoice'], axis=1).head()


,NamaPerusahaan,NamaBarang,Qty
0,PT. BERSAMA SEJAHTERA SAKTI (GAF),OIL SEAL 120MM X 150MM X 14MM,10
1,PT. BERSAMA SEJAHTERA SAKTI (GAF),OIL SEAL 130MM X 160MM X 13MM,10
2,PT. BERSAMA SEJAHTERA SAKTI (GAF),OIL SEAL 150MM X 180MM X 14MM,10
3,PT. SWADAYA ANDIKA (SLF),CAT HAMPLE TAHAN PANAS 600˚C,10
4,PT. BERSAMA SEJAHTERA SAKTI (GAF),ATAP TRANSPARAN 0.8MM X 840MM X 6000MM,70


In [8]:
basket_group = (data.groupby(['Invoice', 'NamaBarang'])['Qty']
                 .sum().unstack().reset_index().fillna(0)
                 .set_index('Invoice'))

# Replace Invoice values with sequential numbers
basket_group.reset_index(inplace=True)
basket_group['Invoice'] = pd.factorize(basket_group['Invoice'])[0] + 1
basket_group.set_index('Invoice', inplace=True)

pd.set_option('display.max_columns', 20)
basket_group

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6588\3228200404.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  basket_group.reset_index(inplace=True)


NamaBarang,PACKING KLINGRITE 5 MM,FITTING LAMP MERCURY,SCUN CABLE 120MM (AL-CU-120),SCUN CABLE 35MM (SC-35),SCUN CABLE 50MM (SC-50),SCUN CABLE 70MM (AL-CU-70),SCUN CABLE 70MM (SC-70),SCUN CABLE 95MM (AL-CU-95),PACKING KLINGRITE 5MM,ELECTRODE LB 52 DIA 4.0MM,...,"MATA BOR 30MM, MERK : NACHI","STANG RIVET, MERK : TEKIRO",BOLA LAMPU 100W,CAP LAMPU TAMAN + TUTUP,"MATA GERGAJI BESI 18-11/4""-062, SANDFLEX",OIL SILICON @18ML,"BESI UNP 100 TEBAL 3,5MM",LAMPU LED 400W(8 X 50W),LAMPU LED 400W (8 X 50W),KIPAS ANGIN MASPION (POWER FAN)
Invoice,,,,,,,,,,,,,,,,,,,,,
1,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,12.0,15.0,15.0,15.0,15.0,15.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Membuat method hot_encode untuk mengubah jumlah beli menjadi membeli(1) atau tidak(0)
def hot_encode(x):
    if(x <= 0):
        return 0
    if(x > 0):
        return 1

In [11]:
# Menerapkan funtion hot_encode
basket_group = basket_group.applymap(hot_encode)
basket_group

# DistribusiBarang = TransactionEncoder().fit_transform(data)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6588\1664427068.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_group = basket_group.applymap(hot_encode)


NamaBarang,PACKING KLINGRITE 5 MM,FITTING LAMP MERCURY,SCUN CABLE 120MM (AL-CU-120),SCUN CABLE 35MM (SC-35),SCUN CABLE 50MM (SC-50),SCUN CABLE 70MM (AL-CU-70),SCUN CABLE 70MM (SC-70),SCUN CABLE 95MM (AL-CU-95),PACKING KLINGRITE 5MM,ELECTRODE LB 52 DIA 4.0MM,...,"MATA BOR 30MM, MERK : NACHI","STANG RIVET, MERK : TEKIRO",BOLA LAMPU 100W,CAP LAMPU TAMAN + TUTUP,"MATA GERGAJI BESI 18-11/4""-062, SANDFLEX",OIL SILICON @18ML,"BESI UNP 100 TEBAL 3,5MM",LAMPU LED 400W(8 X 50W),LAMPU LED 400W (8 X 50W),KIPAS ANGIN MASPION (POWER FAN)
Invoice,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Modelling

In [12]:
# Membangun model menggunakan algoritme Apriori
frq_items = apriori(basket_group, min_support = 0.002, use_colnames=True)

# Membentuk aturan-aturan asosiasi
rules = association_rules(frq_items, metric="confidence", min_threshold=0.3)
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


### Top 5 Apriori

In [13]:
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4,"(BOLT & NUT 3/4"" X 3"")","(BOLT & NUT 5/8"" X 3"")",0.003284,0.003284,0.003284,1.0,304.5,0.003273,inf,1.0
5,"(BOLT & NUT 5/8"" X 3"")","(BOLT & NUT 3/4"" X 3"")",0.003284,0.003284,0.003284,1.0,304.5,0.003273,inf,1.0
16,(SPRAY PAINT PYLOX COLOUR RED),(SPRAY PAINT PYLOX COLOUR BLUE),0.003284,0.003284,0.003284,1.0,304.5,0.003273,inf,1.0
17,(SPRAY PAINT PYLOX COLOUR BLUE),(SPRAY PAINT PYLOX COLOUR RED),0.003284,0.003284,0.003284,1.0,304.5,0.003273,inf,1.0
18,(SPRAY PAINT PYLOX COLOUR WHITE),(SPRAY PAINT PYLOX COLOUR BLUE),0.003284,0.003284,0.003284,1.0,304.5,0.003273,inf,1.0


### Bottom 5 Apriori

In [14]:
rules.tail(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
9,"(BOLT & NUT 3/8"" X 1"")","(BOLT & NUT 3/4"" X 4"")",0.004926,0.004926,0.003284,0.666667,135.333333,0.003260,2.985222,0.997525
41,"(BATU GERINDA POTONG 4"" 100 X 1 X 16MM)",(AMPLAS KERTAS 800),0.006568,0.003284,0.003284,0.500000,152.250000,0.003263,1.993432,1.000000
11,"(BATU GERINDA 4"")",(ANTI KARAT WD40),0.008210,0.003284,0.003284,0.400000,121.800000,0.003257,1.661193,1.000000
12,"(BATU GERINDA 4"")","(BATU GERINDA 6"")",0.008210,0.003284,0.003284,0.400000,121.800000,0.003257,1.661193,1.000000
14,"(BATU GERINDA 4"")","(BATU GERINDA POTONG 4"")",0.008210,0.003284,0.003284,0.400000,121.800000,0.003257,1.661193,1.000000
